In [10]:
import re
import os
from progressbar import ProgressBar
import numpy as np
from sklearn.metrics import precision_recall_fscore_support,f1_score
from collections import Counter as c
from sklearn.model_selection import train_test_split

In [11]:
class topnclass:
    def __init__(self):
        with open("/home/student1/parsefull.csv") as f:
            lin=f.readlines()
        self.con = [x.rstrip('\n') for x in lin]  ####### stripping newlines.........
        self.con =[re.split(':\d+',x) for x in self.con]
    def toplabels(self,n):
        con_new=[x[0]for x in self.con]
        con_new=[list(map(int,re.split('\s',x.replace(",",''))[:-1]))for x in con_new]
        flat_con_new = [item for sublist in con_new for item in sublist]
        label_counts=c(flat_con_new)
        label_top=[x[0]for x in label_counts.most_common(n)]
        return label_top
    def createdatatop(self,n):
        pbar=ProgressBar()
        label_list=self.toplabels(n)
        self.label_list_all=label_list
        ############# discard only labels not data
        tmp=[]
        y=[]###########target class
        X=[]#############documents
        with open('top'+str(n)+'classdata.txt','w') as f:
            for j in pbar(range(len(self.con))):
                tmp=re.split(', ',self.con[j][0])
                count=0
                tmp1=[]
                for x in tmp[:-1]:
                    if int(x) in label_list:
                        count = count+1
                        f.write('__label__')
                        tmp1.append(int(x))
                        f.write(str(x)+' ')
                tmp=tmp[-1].split()
                if int(tmp[0]) in label_list:
                    count=count+1
                    f.write('__label__')
                    tmp1.append(int(tmp[0]))
                    f.write(str(tmp[0])+' ')
                if tmp1:
                    y.append(tmp1)
                if count>0:
                    f.write('w'+str(tmp[1])+' ')
                    tmp2=[]
                    for k in self.con[j][1:-1]:
                        f.write('w'+str(k).replace(' ','')+' ')
                        tmp2.append('w'+str(k).replace(' ',''))
                    X.append(" ".join(tmp2))
                    f.write('\n')
        return X,y
    def fastontop(self,n):
        X,y=self.createdatatop(n)
        X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.30, random_state=30,shuffle=True)
        with open('top'+str(n)+'classtrain.txt','w') as f:
            for i in range(len(X_train)):
                for j in range(len(y_train[i])):
                    f.write("__label__")
                    f.write(str(y_train[i][j])+" ")
                f.write(X_train[i])
                f.write("\n")
        with open('top'+str(n)+'classtest.txt','w') as f:
            for i in range(len(X_test)):
                f.write(X_test[i])
                f.write("\n")
        cmd1="fasttext supervised  -dim 200 -epoch 100 -lr 0.25 -loss hs -input top"+str(n)+"classtrain.txt -output top"+str(n)+"classmodel"
        mod="top"+str(n)+"classmodel.bin "
        tes="top"+str(n)+"classtest.txt "
        res="top"+str(n)+"classresult.txt"
        y_train_labellength=[len(x) for x in y_train]
        Average_labels_per_document = sum(y_train_labellength)/len(y_train_labellength)
        print('Average_labels_per_document in top '+str(n)+' class data is',Average_labels_per_document)
        label_predict_count=str(round(Average_labels_per_document))
        print('So,Predicting ',label_predict_count,'labels using fasttext')
        cmd2="fasttext predict "+mod+ tes+label_predict_count+' > '+ res
        os.system(cmd1)
        os.system(cmd2)
        f = open(res, 'r')###############reading predicted values
        lines =[re.split('__label__', line) for line in f.readlines()]
        f.close()
        lines=[x[1:]for x in lines]
        lines=[list(map(int, x)) if isinstance(x,list) else [int(x)] for x in lines]
        shape=(len(y_test),n)
        y_true=np.zeros(shape,dtype=np.int)
        y_pred=np.zeros(shape,dtype=np.int)
        for x in range(len(y_test)):
            t1=lines[x]
            t2=y_test[x]
            for k in range(len(t1)):
                index1=self.label_list_all.index(t1[k])
                y_pred[x][index1]=1
            for z in range(len(t2)):
                index2=self.label_list_all.index(t2[z])
                y_true[x][index2]=1
        print("For top "+str(n)+" class (precision,recall,fscore) using macro average:",precision_recall_fscore_support(y_true,y_pred, average='macro'))


In [12]:
obj=topnclass()

In [13]:
obj.fastontop(10)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:00:38 Time: 0:00:38


Average_labels_per_document in top 10 class data is 1.2040520887536859
So,Predicting  1 labels using fasttext
For top 10 class (precision,recall,fscore) using macro average: (0.7721319410090449, 0.45440001895511434, 0.544640031905407, None)


In [14]:
obj.fastontop(100)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:01:00 Time: 0:01:00


Average_labels_per_document in top 100 class data is 1.4025902858246426
So,Predicting  1 labels using fasttext
For top 100 class (precision,recall,fscore) using macro average: (0.5063146170884429, 0.2950444835595893, 0.3544236294610442, None)


In [15]:
obj.fastontop(1000)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:03:35 Time: 0:03:35


Average_labels_per_document in top 1000 class data is 1.9115519552849054
So,Predicting  2 labels using fasttext
For top 1000 class (precision,recall,fscore) using macro average: (0.3455700747447608, 0.3395190123331962, 0.3231711861919399, None)


In [16]:
obj.fastontop(10000)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:26:45 Time: 0:26:45


Average_labels_per_document in top 10000 class data is 2.4095504116927997
So,Predicting  2 labels using fasttext
For top 10000 class (precision,recall,fscore) using macro average: (0.2500700256356269, 0.22185674621604384, 0.2148148932757374, None)
